In [1]:
# 장고쉘을 수행하기 위해서는 반드시 실행시켜야 한다.
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [31]:
# 한 건의 측정 데이터를 서버로 전송한다.
import requests, json
##### 다운로드 데이터 ######
tmpData = {
    'dataType' : 'call',
    'phone_no' : 1044700193,
    'meastime' : 202201302315000,
    'networkId' : '5G',
    'groupId' : '16357242412367e98748b7c84601a',
#     'groupId' : '1635724/8333537e98748b7c84601a',
    'currentCount' : 1,
    'ispId' : '45008',
    'testNetworkType' : 'speed',
    'userInfo1' : '서울특별시-금천구-가산동',
    'userInfo2' : '행-용택-1',
    'downloadBandwidth' : 4,
    'uploadBandwidth' : None,
    'latitude' : 37.47653566,
    'longitude' : 126.8913176,
}
jsonData = json.dumps(tmpData)
r = requests.post("http://localhost:8000/monitor/json/", data=jsonData)
r.status_code

500

In [21]:
# 한 건의 측정 데이터를 서버로 전송한다.
import requests, json
##### 업로드 데이터 ######
tmpData = {
    'dataType' : 'call',
    'phone_no' : 1038130003,
    'meastime' : 202201302315000,
    'networkId' : '5G',
    'groupId' : '16357242412367e98748b7c84601a',
#     'groupId' : '16357248333537e98748b7c84601a',
    'currentCount' : 1,
    'ispId' : '45008',
    'testNetworkType' : 'speed',
    'userInfo1' : '서울특별시-금천구-가산동',
    'userInfo2' : '행-용택-1',
    'downloadBandwidth' : None,
    'uploadBandwidth' : 6,
    'latitude' : 37.47653566,
    'longitude' : 126.8913176,
    
}
jsonData = json.dumps(tmpData)
r = requests.post("http://localhost:8000/monitor/json/", data=jsonData)
r.status_code

200

In [29]:
# 해당 위치(위도,경도)에 대한 주소 및 행정동을 조회한다.
from monitor.geo import KakaoLocalAPI
# REST API 키
rest_api_key = "9daef46439c87ea1a53391feb26ebb8b"

kakao = KakaoLocalAPI(rest_api_key)

## Set params
x = 128.06219445
y = 34.9873889

input_coord = "WGS84" # WGS84, WCONGNAMUL, CONGNAMUL, WTM, TM
output_coord = "TM" # WGS84, WCONGNAMUL, CONGNAMUL, WTM, TM

## Request
result = kakao.geo_coord2regioncode(x,y, input_coord, output_coord)
# result['documents'][0]['region_3depth_name']
result

{'meta': {'total_count': 2},
 'documents': [{'region_type': 'B',
   'code': '4824012400',
   'address_name': '경상남도 사천시 노룡동',
   'region_1depth_name': '경상남도',
   'region_2depth_name': '사천시',
   'region_3depth_name': '노룡동',
   'region_4depth_name': '',
   'x': 296184.5342265043,
   'y': 165683.29710986698},
  {'region_type': 'H',
   'code': '4824059500',
   'address_name': '경상남도 사천시 남양동',
   'region_1depth_name': '경상남도',
   'region_2depth_name': '사천시',
   'region_3depth_name': '남양동',
   'region_4depth_name': '',
   'x': 297008.1130364056,
   'y': 164008.47612447804}]}